# 1. 환경 설정 및 라이브러리 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import torch
import torch.nn as nn
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. EDA

## 2-1 train 데이터 확인

In [3]:
train_df = pd.read_csv("/content/drive/MyDrive/모의경진대회/비매너_댓글_식별/data/train.csv", encoding = 'UTF-8-SIG')
train_df

,title,comment,bias,hate
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none
...,...,...,...,...
8362,"""배우 이필립, SNS 스타 연인에게 초호화 프러포즈 눈길""",아니 근데.튜닝한사람은 프러포즈받지도.결혼도못함?ㅋㅋㅋ지들은 돈없어서 못하는것들이ㅋ...,others,hate
8363,"""[SC이슈]""""마약·백스텝·김새롬 탓"""" '실형 피한' 이찬오, 이미지는 치명상(...",그러니깐 여자를 잘만나야되~징글징글한것들 만나면 인생 끝가지 돌아가게 되는듯.. 근...,gender,hate
8364,"""[POP이슈]""""그들만의 세상""""…홍상수♥김민희, 새해데이트에 '반응싸늘'""",참으로 아름다운 커플입니다. 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ...,none,none
8365,[종합] '시크릿 마더' 김소연 누가 죽였나…송윤아와 갈등,재미가 없어요,none,none


In [4]:
# 결측치 확인
train_df.isna().sum()

title      0
comment    0
bias       0
hate       0
dtype: int64

In [5]:
#hate, bias 빈도 확인
pd.crosstab(train_df.bias, train_df.hate, margins=True)

hate,hate,none,All
bias,,,
gender,1216,83,1299
none,2068,3422,5490
others,1437,141,1578
All,4721,3646,8367


## 2-2 Test 데이터 확인

In [6]:
test_df = pd.read_csv("/content/drive/MyDrive/모의경진대회/비매너_댓글_식별/data/test.csv")
test_df

,ID,title,comment
0,0,"류현경♥︎박성훈, 공개연애 4년차 애정전선 이상無..""의지 많이 된다""[종합]",둘다 넘 좋다~행복하세요
1,1,"""현금 유도+1인 1라면?""…'골목식당' 백종원, 초심 잃은 도시락집에 '경악' [종합]",근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데
2,2,"입대 D-11' 서은광의 슬픈 멜로디..비투비, 눈물의 첫 체조경기장[콘서트 종합]",누군데 얘네?
3,3,"아이콘택트' 리쌍 길, 3년 전 결혼설 부인한 이유 공개…""결혼,출산 숨겼다""","쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ..."
4,4,"구하라, 안검하수 반박 해프닝...""당당하다""vs""그렇게까지"" 설전 [종합]",안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...
...,...,...,...
506,506,"[N이슈] 최율, 조재현 성추행 의혹 폭로… 소속사 ""상황 파악 중""",얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?
507,507,"해투4' 이서진, 한지민 '대본 리딩 격리설' 해명…""날씨가 좋아서"" [SC컷]",대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹
508,508,"[SS인터뷰①]박민영 ""'김비서' 행복했다..열애설엔 당당..미소였으니까""",성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...
509,509,"[POP이슈]""사실무근"" 'SKY캐슬' 측 '위올라이' 표절설 부인→여전히 '핫'(종합)",분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...


## 2-3 데이터 전처리
* bias, hate 라벨들의 class를 정수로 변경하여 라벨 인코딩을 하기 위한 딕셔너리입니다.
(bias, hate 컬럼을 합쳐서 하나의 라벨로 만들기) 

In [7]:
# 두 라벨의 가능한 모든 조합 만들기
combinations = np.array(np.meshgrid(train_df.bias.unique(), train_df.hate.unique())).T.reshape(-1,2)
combinations

array([['none', 'none'],
       ['none', 'hate'],
       ['others', 'none'],
       ['others', 'hate'],
       ['gender', 'none'],
       ['gender', 'hate']], dtype=object)

In [8]:
# bias, hate 컬럼을 합친 것
bias_hate = list(np.array([train_df['bias'].values, train_df['hate'].values]).T.reshape(-1,2))
bias_hate[:5]

[array(['none', 'none'], dtype=object),
 array(['none', 'hate'], dtype=object),
 array(['others', 'hate'], dtype=object),
 array(['none', 'none'], dtype=object),
 array(['none', 'none'], dtype=object)]

In [9]:
#train_data에 라벨 합친 것을 대입
labels = []
for i, arr in enumerate(bias_hate):
    for idx, elem in enumerate(combinations):
        if np.array_equal(elem, arr):
            labels.append(idx)

train_df['label'] = labels
train_df.head()

,title,comment,bias,hate,label
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none,0
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,1
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate,3
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none,0
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none,0


## 3. TF-IDF 테스트
* comment만 보여줘야하니 title 컬럼 삭제해야함

In [10]:
#train 데이터 title 삭제
train_df2 = train_df.drop(["title"], axis=1)
train_df2

,comment,bias,hate,label
0,김태리 정말 연기잘해 진짜,none,none,0
1,공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,1
2,누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate,3
3,일본 축구 져라,none,none,0
4,난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none,0
...,...,...,...,...
8362,아니 근데.튜닝한사람은 프러포즈받지도.결혼도못함?ㅋㅋㅋ지들은 돈없어서 못하는것들이ㅋ...,others,hate,3
8363,그러니깐 여자를 잘만나야되~징글징글한것들 만나면 인생 끝가지 돌아가게 되는듯.. 근...,gender,hate,5
8364,참으로 아름다운 커플입니다. 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ...,none,none,0
8365,재미가 없어요,none,none,0


In [11]:
#test 데이터 title 삭제
test_df2 = test_df.drop(["title"], axis=1)
test_df2

,ID,comment
0,0,둘다 넘 좋다~행복하세요
1,1,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데
2,2,누군데 얘네?
3,3,"쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ..."
4,4,안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...
...,...,...
506,506,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?
507,507,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹
508,508,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...
509,509,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...


In [12]:
!pip install rich

     |████████████████████████████████| 217 kB 7.2 MB/s 
     |████████████████████████████████| 51 kB 7.7 MB/s 


In [17]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.1 MB/s 


In [18]:
import pandas as pd
import numpy as np
import os
import re
import rich  # 출력을 예쁘게 꾸며주는 라이브러리
from rich.table import Table

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from tqdm.auto import tqdm

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

### null값, 공백 제거, 한글과 공백 이외의 문자 제거

In [14]:
# 두 칸 이상의 빈칸을 한칸으로 바꿔줌
train_df2["comment"]  = train_df2["comment"].str.replace("\s+", " ", regex=True)
test_df2["commnet"]  = test_df2["comment"].str.replace("\s+", " ", regex=True)

In [15]:
#가장 먼저 한글과 공백만 남긴 뒤 preprocessed라는 컬럼으로 저장
def preprocess(text):
  text = text.lower()
  text = re.sub("[^A-Za-z가-힣 ]","", text)  # re.sub(r"[^A-Za-zㄱ-ㅎㅏ-ㅣ가-힣 ]","", text)
  return text

train_df2['preprocessed'] = train_df2.comment.apply(lambda x : preprocess(x))
test_df2['preprocessed'] = test_df2.comment.apply(lambda x : preprocess(x))

train_df2.head()

,comment,bias,hate,label,preprocessed
0,김태리 정말 연기잘해 진짜,none,none,0,김태리 정말 연기잘해 진짜
1,공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,1,공효진 발연기나이질생각이읍던데 왜계속주연일까
2,누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate,3,누구처럼 돈만 밝히는 저급인생은 살아가지마시길 행복은 머니순이 아니니깐 작은거에 감...
3,일본 축구 져라,none,none,0,일본 축구 져라
4,난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none,0,난 절대로 임현주 욕하는인간이랑은 안논다


### TFIDF 함수 설정 및 Model 적용
* 전처리된 것보다 전처리 안된 comment가 높은 결과가 나와서 그냥 전처리안된걸로 하기로함

In [16]:
#TfidfVectorizer와 모델을 연결한 파이프라인을 반환하는 함수
def get_pipe(model, model_name: str) -> Pipeline:
    tfidf = TfidfVectorizer(analyzer="char", ngram_range=(1, 3))
    pipe = Pipeline([
        ("tfidf", tfidf),
        (model_name, model)
    ])
    return pipe

In [31]:
#  KFold 예측 후 F1-score를 반환하는 함수
def return_kfold_score(model, k: int = 5) -> float:
    kfold = StratifiedKFold(k, shuffle=True, random_state=42)
    result = []
    for train_idx, test_idx in kfold.split(train_df2["comment"], train_df2["label"]):
        train, val = train_df2.iloc[train_idx], train_df2.iloc[test_idx]
        model.fit(train["comment"], train["label"])
        pred = model.predict(val["comment"])
        score = f1_score(val["label"], pred, average='micro')
        result.append(score)

    return np.mean(result)

In [32]:
#모델 별로 성능 확인
models = [
    ('SVC', SVC()),
    ("SGD", SGDClassifier(random_state=42, n_jobs=-1)),
    ("logistic", SGDClassifier(loss="log", random_state=42, n_jobs=-1)),
    ("pac", PassiveAggressiveClassifier(random_state=42, n_jobs=-1)),
    ("lgbm", LGBMClassifier(random_state=42, tree_method= 'gpu_hist')),
    ("xgb", XGBClassifier(random_state=42)),
    # ("cat", CatBoostClassifier(random_state=42))
]

model_pipes = [(name, get_pipe(model, name)) for name, model in models]

In [33]:
#모델 별로 학습 후 F1-score 출력
table = Table(title="Model Comparison Table")
table.add_column("Model Name", justify="left", style="green")
table.add_column("F1-Score", justify="right")

for model_name, model in tqdm(model_pipes, leave=False):
    score = return_kfold_score(model)
    table.add_row(model_name, f"{score:0.3f}")

rich.print(table)

  0%|          | 0/6 [00:00<?, ?it/s]

 Model Comparison Table  
┏━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Model Name ┃ F1-Score ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━┩
│ SVC        │    0.539 │
│ SGD        │    0.573 │
│ logistic   │    0.566 │
│ pac        │    0.565 │
│ lgbm       │    0.563 │
│ xgb        │    0.535 │
└────────────┴──────────┘

### 3. 하이퍼파라미터 탐색

In [44]:
# !pip install scikit_optimize

In [43]:
# # 하이퍼파라미터 탐색
# from skopt import BayesSearchCV
# from skopt.space import Categorical, Integer, Real

# space = {
#     'SVC': {"SVC__C": Real(0, 1)},
#     "SGD": {"SGD__alpha": Real(1e-5, 1, prior="log-uniform"),
#             "SGD__loss": Categorical(["hinge", "modified_huber"]),
#             "SGD__tol": Real(1e-4, 1, prior="log-uniform")},
#     "logistic": {"logistic__alpha": Real(1e-6, 1, prior="log-uniform"),
#                  "logistic__tol": Real(1e-4, 1, prior="log-uniform")},
#     "pac": {"pac__C": Real(0, 1),
#             "pac__tol": Real(1e-3, 1, prior="log-uniform")},
#     "lgbm": {"lgbm__n_estimators": Integer(0, 50),
#              "lgbm__learning_rate": Real(1e-3, 1, prior="log-uniform"),
#              "lgbm__reg_lambda": Real(0, 0.5)},
#     "xgb": {"xgb__n_estimators": Integer(0, 50),
#              "xgb__learning_rate": Real(1e-3, 1, prior="log-uniform"),
#              "xgb__reg_lambda": Real(0, 0.5)}
# }

# def bayes_search_cv(model, model_name: str):
#     opt = BayesSearchCV(
#             model, space[model_name], scoring="f1_micro", n_iter=30, n_jobs=-1, random_state=42
#         )
#     opt.fit(train_df2["comment"], train_df["label"])
#     return opt

In [42]:
# best_models = []
# opts = []

# for name, model in tqdm(model_pipes):
#     opt = bayes_search_cv(model, name)
#     opts.append((name, opt))
#     best_models.append((name, opt.best_estimator_))

In [ ]:
# #최적의 파라미터 출력
# best_models

In [ ]:
# from sklearn.model_selection import cross_val_score

# table = Table(title="Model Comparison Table")
# table.add_column("Model Name", justify="left", style="green")
# table.add_column("F1-Score", justify="right")

# for model_name, model in tqdm(best_models, leave=False):
#     score = cross_val_score(
#         model, train_df["comment"], train_df["label"], scoring="f1_micro", n_jobs=-1
#     )
#     score = np.mean(score)
#     table.add_row(model_name, f"{score:0.4f}")

# rich.print(table)

In [45]:
from sklearn.ensemble import StackingClassifier

stack_models = [(name, get_pipe(model, name)) for name, model in models]

stacking = StackingClassifier(stack_models)
score = return_kfold_score(stacking)
rich.print(score)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.5877855546771731

In [46]:
stacking.fit(train_df2['comment'], train_df2['label'])
submission_pred = stacking.predict(test_df2['comment'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [47]:
submission_pred

array([0, 0, 1, 1, 0, 3, 1, 3, 5, 0, 0, 5, 0, 0, 0, 5, 0, 0, 0, 0, 5, 3,
       3, 1, 0, 3, 3, 0, 0, 0, 3, 5, 0, 3, 0, 5, 0, 0, 0, 0, 5, 0, 3, 0,
       0, 1, 1, 0, 0, 5, 0, 0, 1, 3, 0, 5, 5, 0, 3, 1, 1, 5, 0, 5, 0, 0,
       0, 1, 0, 5, 0, 1, 1, 1, 1, 0, 5, 0, 5, 1, 1, 1, 3, 0, 0, 0, 0, 3,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 3, 1, 1, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 5, 1, 5, 5, 0, 3, 0, 1,
       0, 5, 1, 0, 1, 0, 0, 1, 0, 1, 5, 0, 0, 0, 3, 1, 1, 0, 3, 0, 5, 5,
       5, 0, 5, 3, 3, 0, 0, 5, 0, 5, 5, 5, 0, 1, 5, 0, 5, 0, 0, 4, 5, 1,
       5, 0, 5, 0, 0, 3, 5, 1, 5, 0, 0, 3, 0, 0, 1, 0, 3, 1, 1, 0, 5, 3,
       0, 0, 0, 1, 3, 1, 5, 1, 0, 0, 1, 3, 3, 3, 0, 0, 0, 0, 5, 0, 1, 0,
       1, 0, 5, 1, 1, 1, 3, 0, 0, 0, 5, 0, 1, 0, 5, 5, 5, 0, 0, 3, 5, 1,
       5, 0, 0, 0, 1, 0, 0, 0, 3, 0, 1, 0, 3, 0, 0, 0, 5, 5, 0, 1, 5, 0,
       0, 0, 5, 5, 1, 0, 1, 5, 0, 0, 0, 0, 0, 0, 1, 3, 3, 5, 5, 0, 0, 0,
       1, 5, 0, 0, 1, 1, 1, 0, 3, 0, 1, 3, 0, 0, 5,

In [49]:
# 0-5 사이의 라벨 값 별로 bias, hate로 디코딩 하기 위한 딕셔너리
bias_dict = {0: 'none', 1: 'none', 2: 'others', 3:'others', 4:'gender', 5:'gender'}
hate_dict = {0: 'none', 1: 'hate', 2: 'none', 3:'hate', 4:'none', 5:'hate'}

# 인코딩 값으로 나온 타겟 변수를 디코딩
pred_bias = ['' for i in range(len(submission_pred))]
pred_hate = ['' for i in range(len(submission_pred))]

for idx, label in enumerate(submission_pred):
    pred_bias[idx]=(str(bias_dict[label]))
    pred_hate[idx]=(str(hate_dict[label]))
print('decode Completed!')

decode Completed!


In [50]:
submission = pd.read_csv('/content/drive/MyDrive/모의경진대회/비매너_댓글_식별/data/sample_submission.csv')
submission['bias'] = pred_bias
submission['hate'] = pred_hate
submission

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,none,hate
4,4,none,none
...,...,...,...
506,506,none,hate
507,507,none,none
508,508,others,hate
509,509,none,none


In [51]:
submission.to_csv('/content/drive/MyDrive/모의경진대회/비매너_댓글_식별/data2/NLP_Machinelearning3.csv')